# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
import itertools
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('/Users/dasakalinina/Desktop/Practicum/_all_projects/Loan_owners/data.csv')

display(data.head(10))
print(data.info())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


In [3]:
print('Общее число дубликатов:', data.duplicated().sum())
#проверить пропуски
print(data.isna().sum())
#посмотреть уникальные значения по каждому столбцу
print('Значения в children:', data['children'].sort_values().unique())
print('Значения в dob_years:', data['dob_years'].sort_values().unique())
print('Значения в education:', data['education'].sort_values().unique())
print('Значения в family_status:', data['family_status'].sort_values().unique())
print('Значения в gender:', data['gender'].unique())
print('Значения в income_type:', data['income_type'].sort_values().unique())
print('Значения в debt:', data['debt'].unique())
print('Значения в purpose:', data['purpose'].sort_values().unique())

Общее число дубликатов: 54
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
Значения в children: [-1  0  1  2  3  4  5 20]
Значения в dob_years: [ 0 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75]
Значения в education: ['ВЫСШЕЕ' 'Высшее' 'НАЧАЛЬНОЕ' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Начальное'
 'Неоконченное высшее' 'СРЕДНЕЕ' 'Среднее' 'УЧЕНАЯ СТЕПЕНЬ'
 'Ученая степень' 'высшее' 'начальное' 'неоконченное высшее' 'среднее'
 'ученая степень']
Значения в family_status: ['Не женат / не замужем' 'в разводе' 'вдовец / вдова' 'гражданский брак'
 'женат / замужем']
Значения в gender: ['F' 'M' 'XNA']
Значения в income_type

**Вывод**

Таблица состоит из нескольких столбцов с разными типами значений: от целочисленных и вещественных до текстовых. 

**Количественные переменные:**
* children,
* days_employed,
* dob_years,
* debt,
* total_income

**Категориальные переменные:**
* education,
* family_status,
* gender,
* income_type,
* purpose.

**Булевые переменные:**
* education_id,
* family_status,
* debt.

В ряде столбцов есть **пропуски данных** - это столбцы:
- days_employed (общий трудовой стаж в днях),
- total_income (ежемесячный доход).

**Общее число пропущенных значений в каждом столбце - 2174.**

Также пропуски есть в столбце 'Gender', но в виде текстовых значений.

После просмотра массива данных и с учетом совпадающего количества по пропущенным данным есть предположение, что пропуски в обоих типах данных у людей, которые не трудоустроены, соответственно, у них нет ни трудового стажа, ни дохода.

Интересно, что трудовой стаж в днях измеряется в виде числа float64, а не int. Возможно, учет трудового стажа идет включая время визита клиента в банк. Вызывает вопрос отображение трудового стажа в виде отрицательной величины у первых 9 клиентов, кроме пенсионера.

**Всего дубликатов в таблице - 54 штуки.**

**После проверки на уникальность выявлено:**
- большое количество разных значений в столбцах 'Цель покупки жилья' (привести к единому виду через лемматизацию),
- разные варианты написания образования (требуется замена),
- интересные значения в столбце 'Количество детей' (отрицательное и 20),
- в значения столбца 'Пол' есть строки с пропусками,
- в столбце dob_years присутствует значение возраста равное 0 (пропуск данных).

**Т.к. в целях исследования задачи проверить:**
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

То в дальнейшем буду **работать со столбцами, которые нужны для целей исследования**, а именно:
- children,
- family_status,
- total_income,
- purpose,
- debt.


## Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
# проверяю начало и конец таблицы с
display(data[data['total_income'].isna()].head()) 
display(data[data['total_income'].isna()].tail()) 
#еще раз подсчет общего количества пропусков
print(len(data[data['total_income'].isna()]))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля


2174


Значения отсутствуют одновременно в столбцах: Трудовой стаж и Общий доход. Но при этом заполнен столбец Источник дохода, так что предыдущий вывод об отсутствии работы неверный, следовательно, требуется заполнение данными.

In [5]:
len(data[data['total_income'].isna() & data['days_employed'].isna()])

2174

In [6]:
#Рассчитываю медианное значение дохода для каждой группы по типу дохода
print(data.groupby('income_type')['total_income'].median())

#Рассчитываю общий медианный доход для всех групп
print ('Медианный доход:', data['total_income'].median())

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64
Медианный доход: 145017.93753253992


Заменяю пустые значения зарплаты на медианные.

In [7]:
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))

#проверка
display(data[data['total_income'].isna()].tail()) 
print(len(data[data['total_income'].isna()]))
display(data['total_income'].value_counts())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


0


142594.396847    1105
172357.950966     509
118514.486412     414
150447.935283     147
499163.144947       2
                 ... 
148042.721049       1
60039.334460        1
175979.762960       1
155819.968351       1
82047.418899        1
Name: total_income, Length: 19353, dtype: int64

In [8]:
data['income_type_id'] = data['income_type'].map(
    {'студент': 0, 'безработный': 1, 'пенсионер': 2,
                    'в декрете':3, 'госслужащий': 4, 'сотрудник':5, 'компаньон':6, 'предприниматель':7} )
data.corr()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income,income_type_id
children,1.000000,-0.138354,-0.174358,-0.012695,-0.091284,0.018470,0.018485,0.123070
days_employed,-0.138354,1.000000,0.582643,0.080565,0.005726,-0.047110,-0.136648,-0.917710
dob_years,-0.174358,0.582643,1.000000,0.067467,-0.069988,-0.069620,-0.058062,-0.544197
education_id,-0.012695,0.080565,0.067467,1.000000,0.007876,0.052906,-0.171804,-0.090168
family_status_id,-0.091284,0.005726,-0.069988,0.007876,1.000000,0.020611,-0.008371,0.007561
debt,0.018470,-0.047110,-0.069620,0.052906,0.020611,1.000000,-0.012304,0.035635
total_income,0.018485,-0.136648,-0.058062,-0.171804,-0.008371,-0.012304,1.000000,0.184124
income_type_id,0.123070,-0.917710,-0.544197,-0.090168,0.007561,0.035635,0.184124,1.000000


In [9]:
#проверяю странное значение по полу
missed_gender = data[data['gender']=='XNA']
display (missed_gender)

display(data['gender'].value_counts())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_id
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости,6


F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В поле Gender отсутствует всего 1 значение из 21525, соотвественно, им можно принебречь и удалить строку. Существенного влияния на результаты она не окажет.
Либо с учетом того, что в целях исследования параметр пола не учитывается, а большая часть заемщиков - женщины, можно скорректировать пол на женский. Таким образом, останутся данные о количестве детей, семейном положении, уровне дохода и целях кредита. 
Второй вариант какжется более предпочтительным.

In [10]:
#произвожу замену неизвестного значения пола на женский и проверяю
data.loc[data['gender'] =='XNA', 'gender'] = 'F'
display(data['gender'].value_counts())

F    14237
M     7288
Name: gender, dtype: int64

### Замена типа данных

В столбце children есть отрицательные значения, скорее всего, это просто техническая ошибка, поэтому можно заменить значения в этом столбце на такие же по модулю.
Также сомнительно наличие детей в количестве 20 человек, скорее всего, поставлен лишний 0, соответственно, можно произвести замену на 2.

In [11]:
data['children'] = data['children'].abs()
data.loc[data['children'] == 20, 'children'] = 2
print('Значения в children:', data['children'].unique())
print(data['children'].value_counts())

Значения в children: [1 0 3 2 4 5]
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


Заменим значения в столбце Total_income на целочисленный.

In [12]:
data['total_income'] = data['total_income'].astype('int')
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_id
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,5
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,5
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,5
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,2


**Вывод**

С помощью функции .astype была произведена замена float64 на int (целочисленный).
С помощью функции .abs были изменены отрицательные значения в столбце children на такие же по модулю.
С использованием логической индексации была произведена замена количества детей 20 на 2.

### Обработка дубликатов

In [13]:
print('Общее число дубликатов:', data.duplicated().sum())

Общее число дубликатов: 54


In [14]:
print (data['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


Т.к. варианты написания образования очень разнообразны, хотя суть одна и та же, возможно, есть еще дубликаты. Чтобы их найти приведем образование к единому виду.

In [15]:
data['education'] = data['education'].str.lower()
print (data['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


Повторно проверим количество дубликатов.

In [16]:
print('Общее число дубликатов:', data.duplicated().sum())

Общее число дубликатов: 71


Удалим дубликаты и произведем проверку удаления.

In [17]:
data = data.drop_duplicates()
data = data.dropna().reset_index(drop=True)
print('Общее число дубликатов:', data.duplicated().sum())

Общее число дубликатов: 0


**Вывод**

За счет приведения образования к общему виду с помощью функции str.lower (все строчные буквы) удалось выявить дополнительно 16 дубликатов и удалить их вместе с основным массивом.
Возможная причина возникновения дубликатов - технические ошибки выгрузки данных или ошибочное дублирование в системе.

### Лемматизация

In [18]:
m = Mystem()

print(data['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'жилье' 'операции со своей недвижимостью' 'автомобили'
 'заняться образованием' 'получение образования'
 'сделка с подержанным автомобилем' 'автомобиль' 'свадьба'
 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'строительство жилой недвижимости' 'ремонт жилью'
 'заняться высшим образованием']


In [19]:
#создадим функцию, которая будет производить построчную лемматизацию столбца purpose и в зависимости от наличия определенных ключевых слов относить строку к той или иной категории
def create_category_purpose(row):
    lem_purpose = m.lemmatize(row['purpose'])
    if 'автомобиль' in lem_purpose:
        return 'автомобиль'
    if ('жилье' in lem_purpose) or ('недвижимость' in lem_purpose):
        return 'недвижимость'
    if 'образование'in lem_purpose:
        return 'образование'
    if 'свадьба' in lem_purpose:
        return 'свадьба'
#добавим новый столбец 'purpose_category', куда внесем категории, кроторый определили на предыдущем шаге
data['purpose_category'] = data.apply(create_category_purpose, axis=1)

In [20]:
#проверка предыдущего шага
display(data.head(15))
print(data['purpose_category'].unique())
print(data['purpose_category'].value_counts())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_id,purpose_category
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,5,недвижимость
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,5,автомобиль
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5,недвижимость
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,5,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,2,свадьба
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,6,недвижимость
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,6,недвижимость
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,5,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,5,свадьба
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,5,недвижимость


['недвижимость' 'автомобиль' 'образование' 'свадьба']
недвижимость    9758
автомобиль      3897
образование     3597
свадьба         2099
Name: purpose_category, dtype: int64


**Вывод**

Был добавлен новый столбец с категорией оформления кредита по следующему алгоритму:

1) произведена лемматизация в столбце purpose по каждой строке,

2) при наличии ключевых слов производилось отнесение их в определенную 'категорию' перебор осуществляется посредством функции if - return,

3) с помощью функции .apply полученные новые значения были добавлены в новый столбец purpose_category.

### Категоризация данных

In [21]:
#проверка, если есть встречающиеся значения
print(data['total_income'].value_counts()) 

172040    3
97012     3
204827    3
133122    3
102816    3
         ..
381117    1
70100     1
110461    1
102836    1
82047     1
Name: total_income, Length: 18606, dtype: int64


In [22]:
#медианный доход по каждой группе (повторно для удобства)
print(data.groupby('income_type')['total_income'].median())
#медиоанный доход для всех групп
print ('Медианный доход:', data['total_income'].median())

income_type
безработный        131339.0
в декрете           53829.0
госслужащий        150447.5
компаньон          172357.0
пенсионер          118514.0
предприниматель    499163.0
сотрудник          142594.0
студент             98201.0
Name: total_income, dtype: float64
Медианный доход: 145017.0


In [23]:
#создание функции для отнесения дохода в определенную группу в зависимости от размера total_income
def income_group(total_income):
    if total_income <= 100000:
        return 'низкий доход'
    if total_income <= 200000:
        return 'средний доход'
    return 'высокий доход'

#проверка
print(income_group(53829))
print(income_group(150447))
print(income_group(499163))


низкий доход
средний доход
высокий доход


In [24]:
#добавление нового столбца с группой по доходу
data['income_group'] = data['total_income'].apply(income_group)
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_id,purpose_category,income_group
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,5,недвижимость,высокий доход
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,5,автомобиль,средний доход
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5,недвижимость,средний доход
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,5,образование,высокий доход
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,2,свадьба,средний доход
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,6,недвижимость,высокий доход
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,6,недвижимость,высокий доход
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,5,образование,средний доход
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,5,свадьба,низкий доход
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,5,недвижимость,средний доход


In [25]:
#подсчет количества уникальных значений в каждой группе
display(data['income_group'].value_counts())

средний доход    9822
высокий доход    5066
низкий доход     4463
Name: income_group, dtype: int64

In [26]:
#подсчет значений по количеству детей
print(data['children'].value_counts())

0    12710
1     4387
2     1918
3      294
4       34
5        8
Name: children, dtype: int64


In [27]:
#создание функции для отнесения в определенную группу в зависимости от наличия детей
def child_group(children):
    if children == 0:
        return 'нет детей'
    if children > 0:
        return 'есть дети'

#проверка
print(child_group(0))
print(child_group(3))

нет детей
есть дети


In [28]:
#добавление нового столбца с группой по наличию детей
data['child_group'] = data['children'].apply(child_group)
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_id,purpose_category,income_group,child_group
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,5,недвижимость,высокий доход,есть дети
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,5,автомобиль,средний доход,есть дети
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,5,недвижимость,средний доход,нет детей
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,5,образование,высокий доход,есть дети
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,2,свадьба,средний доход,нет детей
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,6,недвижимость,высокий доход,нет детей
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,6,недвижимость,высокий доход,нет детей
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,5,образование,средний доход,нет детей
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,5,свадьба,низкий доход,есть дети
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,5,недвижимость,средний доход,нет детей


In [29]:
#подсчет количеств в разных категориях по количеству детей
display(data['child_group'].value_counts())

нет детей    12710
есть дети     6641
Name: child_group, dtype: int64

**Вывод**

Категоризация данных по целям кредитования была произведена ранее (в разделе Лемматизация).

Категоризация по уровню дохода была сделана с созданием функции, которая посредством if - return последовательно проверяет в какой диапазон попадает определенные значение total_income и в зависимости от этого относит значение в определенную категорию.

Была сделана проверка функции перебором различных значений.

Далее посредством функции .apply был создан новый столбец с категорией дохода. С помощью функции value_counts были рассчитано количество той или иной категории, по полученным результатам получилось, что людей со средним доходом больше всего, что кажется логичным (как бедных, так и богатых людей в идеале не должно быть много).

Аналогично была произведена категоризация по наличию детей в семье.

* Категоризация данных по целям кредитования была произведена ранее (в разделе Лемматизация).

* Категоризация по уровню дохода была сделана с созданием функции, которая:
    * Посредством if - return последовательно проверяет в какой диапазон попадает определенные значение total_income и в зависимости от этого относит значение в определенную категорию.

    * Была сделана проверка функции перебором различных значений.

    * Далее посредством функции .apply был создан новый столбец с категорией дохода. С помощью функции value_counts были рассчитано количество той или иной категории, по полученным результатам получилось, что людей со средним доходом больше всего, что кажется логичным (как бедных, так и богатых людей в идеале не должно быть много).

* Аналогично была произведена категоризация по наличию детей в семье.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [30]:
#создание сводной таблицы по наличию детей и наличию долга с подсчетом количества строк в каждой категории
pivot_table_child_group = data.pivot_table(index=['child_group'], columns='debt', values='total_income', aggfunc='count')
display(pivot_table_child_group)

debt,0,1
child_group,,
есть дети,6022,619
нет детей,11758,952


In [31]:
#расчет доли заемщицов с долгом по каждой из категорий (есть дети / нет детей) и добавления столбца с расчетом в сводную таблицу
pivot_table_child_group['percent'] = pivot_table_child_group[1]/(pivot_table_child_group[1] + pivot_table_child_group[0]) * 100
display(pivot_table_child_group)

debt,0,1,percent
child_group,,,
есть дети,6022,619,9.320885
нет детей,11758,952,7.490165


**Вывод**

У заемщиков с детьми вероятность возникновения долга по кредиту выше, чем у заемщиков без детей.

In [32]:
#расчет доли заемщицов с долгом по всем категориям и добавления столбца с расчетом в сводную таблицу
#создание сводной таблицы по наличию детей и наличию долга с подсчетом количества строк в каждой категории
pivot_table_debt_share = data.pivot_table (columns = 'debt', values='total_income', aggfunc='count')
pivot_table_debt_share['percent_debt'] = pivot_table_debt_share[1]/(pivot_table_debt_share[1] + pivot_table_debt_share[0]) * 100
display(pivot_table_debt_share)

debt,0,1,percent_debt
total_income,17780,1571,8.118443


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [33]:
#создание сводной таблицы по семейному положению и наличию долга с подсчетом количества строк в каждой категории
pivot_table_child_group['percent'] = pivot_table_child_group[1] /(pivot_table_child_group[1] + pivot_table_child_group[0]) * 100
display(pivot_table_child_group)

debt,0,1,percent
child_group,,,
есть дети,6022,619,9.320885
нет детей,11758,952,7.490165


In [34]:
#создание сводной таблицы по семейному положению
pivot_table_status_group = data.pivot_table(index=['family_status'], columns='debt', values='total_income', aggfunc='count')
#расчет доли заемщицов с долгом по каждой из категорий по семейному положению и добавления столбца с расчетом в сводную таблицу
pivot_table_status_group['percent'] = pivot_table_status_group[1]/(pivot_table_status_group[1] + pivot_table_status_group[0]) * 100
display(pivot_table_status_group)

debt,0,1,percent
family_status,,,
Не женат / не замужем,2271,254,10.059406
в разводе,1007,76,7.017544
вдовец / вдова,809,56,6.473988
гражданский брак,3396,339,9.076305
женат / замужем,10297,846,7.592210


**Вывод**

Максимальная вероятность возникновения долга у неженатых/незамужих заемщиков, следующие за ними - заемщики, состоящие в гражданском браке, наименьшая вероятность долга у овдовевших. Зависимость прослеживается.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [35]:
#создание сводной таблицы, расчет доли заемщицов с долгом по каждой из категорий по уровню дохода и добавления столбца с расчетом в сводную таблицу
pivot_table_income_group = data.pivot_table(index=['income_group'], columns='debt', values='total_income', aggfunc='count')
pivot_table_income_group['percent'] = pivot_table_income_group[1]/(pivot_table_income_group[1] + pivot_table_income_group[0]) * 100
display(pivot_table_income_group)

debt,0,1,percent
income_group,,,
высокий доход,4708,358,7.066719
низкий доход,4109,354,7.931884
средний доход,8963,859,8.745673


**Вывод**

Зависимость есть: наименьшая вероятность возникновения долга у заемщиков с высоким доходом, наибольшия - со средним уровнем дохода.

- Как разные цели кредита влияют на его возврат в срок?

In [36]:
#создание сводной таблицы, расчет доли заемщицов с долгом по каждой из категорий по цели кредита и добавления столбца с расчетом в сводную таблицу
pivot_table_purpose_category = data.pivot_table(index=['purpose_category'], columns='debt', values='total_income', aggfunc='count')
pivot_table_purpose_category['percent'] = pivot_table_purpose_category[1]/(pivot_table_purpose_category[1] + pivot_table_purpose_category[0]) * 100
display(pivot_table_purpose_category)

debt,0,1,percent
purpose_category,,,
автомобиль,3530,367,9.417501
недвижимость,9043,715,7.327321
образование,3266,331,9.202113
свадьба,1941,158,7.527394


**Вывод**

Наименьшая вероятность долга у заемщиков, оформивших кредит на проиобретение недвижимости или свадьбу, наибольшая - при кредите на автомобиль или образование.

## Шаг 4. Общий вывод

При принятии решения о выдаче кредита необходимо принимать во внимание и семейное положение, и наличие детей, и цель кредита, и уровень дохода заемщиков.